# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма.

Цель проекта - выполнить предобработку данных и изучить их, чтобы найти интересные особенности и зависимости, которые существуют на рынке недвижимости. 

Для этого нам предстоит изучить имеющиеся исходные данные, проверить таблицу на наличие пропусков, соответствия типа данных и выявить неявные дубли. Затем необходимо проверить данные на наличие аномальных и редких значений и избавиться от них. Затем добавить в в таблицу столбцы со значениями, которые нам понадобятся в исследовании, например, цена одного квадратного метра, 
тип этажа квартиры, расстояние в км до центра города и т.д.

На этапе исследования с помощь графиков изучим содержание каждого столбца, рассчитаем необходимые параметры, такие как скорость продажи квартиры и приступаем к выявлению факторов, влияющих на стоимость квартиры. С помощью диаграмм и коэффициента корреляции изучим такие параметры как общая площадь, количество комнат, день появления объявления и т.д. Рассчитаем необходимые новые параметры такие как населенные пункты по количеству объявлений и среднюю цену каждого километра. Выявленные закономерности и факторы влияния отразим в выводе.  


### Откройте файл с данными и изучите общую информацию. 

In [1]:
#Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt

#Чтение файла и разделением колонок с помощью табуляции 
data = pd.read_csv('/datasets/real_estate_data.csv', sep = '\t')

#Вывод первых 20 строк датафрейма для ознакомления
data.head(20)


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

***Вывод***

В датафрейме встречаются пропуски по столбцам, например столбец 'is_apartment' практически полностью состоит из NaN.

In [ ]:
#приведение наименования столбца к единому регистру
data.rename(columns = {'cityCenters_nearest':'city_centers_nearest'}, inplace = True )

In [ ]:
#Вывод общей информации о датафрейме
data.info()

***Вывод***
Метод info() подтверждает, что полностью заполненных столбцов всего 8 из 21.

In [ ]:
#Постройка гистограмм для всех числовых столбцов
data.hist(figsize=(15,20))

**Вывод**

Гистограммы по столбцам 'last_price','ceiling_height' имнют одинаковый вид - один столбец. В гистограмме по 'balcony' встречаются значения о 2,3 и 4 балконах в квартире. 

### Предобработка данных

In [ ]:
#Вывод пропущенных значений для каждого столбца

data.isna().sum().sort_values(ascending=False)

***Вывод***

Можно заменить пропущенные значения в следующих столбцах: 

**'balcony'**(11519 пропусков)- если не указанно количество балконов, то скорей всего их нет, пропуски можно заменить на 0  

**parks_around3000** (15620) и **ponds_around3000**(5518 пропусков) - мы не можем заменить пропуски в столбце количество парков без дополнительных данных, но мы можем заменить на 0 пропуски в столбце расстояние до ближайшего парка, там где стоит 0 в столбце 'ponds_around3000'. Тоже самое можно сделать со столбцами **parks_nearest** и **ponds_nearest** 

**is_apartment** (20924 пропусков) - если не указанно является ли объект апартаментами, то скорей всего он ими не является, пропуски можно заменить на значение False

**locality_name** (49 пропусков) и **floors_total**(86 пропусков) - в данных столбцах незначительные пропуски и сами мы не можем их заменить, т.к. не можем знать ни название населенного пункта, ни количество этажей в доме. Эти пропуски можно удалить 

Заполнить пропуски в столбцах с расстоянием до центра города и до аэропорта без дополнительных данных и уточнений не получится. Так же как и пропуски в столбцах с количеством дней объявления (возможно техническая ошибка), с данными по площади кухни и жилой площади и высотой потолков. 

Таким образом мы мы уберем из датафрейма самые большие пропуски

In [ ]:
#Замена пропущенных значений в столбце 'balcony' на 0

data['balcony'] = data['balcony'].fillna(0)

#Замена пропущенных значений в столбце 'parks_around3000' на 0


#Замена пропущенных значений в столбце 'parks_nearest' на 0, там где значение в столбце 'parks_around3000' равно 0

data.loc[(data['parks_around3000']== 0), ('parks_around3000', 'parks_nearest')] = 0

##Замена пропущенных значений в столбце 'ponds_around3000' на 0, там где значение в столбце 'ponds_around3000' равно 0

data.loc[(data['ponds_around3000']== 0), ('ponds_around3000', 'ponds_nearest')] = 0

#Замена пропущенных значений в столбце 'is_apartment' на False

data['is_apartment'] = data['is_apartment'].fillna(False)

#Удаление пропусков в столбце 'locality_name'
data = data.dropna(subset=['locality_name'])

#Удаление пропусков в столбце 'floors_total'
data = data.dropna(subset=['floors_total'])

In [ ]:
#Выводим оставшиеся пропуски
data.isna().sum()

**Вывод**

Заполнить пропуски в столбцах с расстоянием до центра города и до аэрапорта без дополнительных данных и уточнений не получится. Так же как и пропуски в столбцах с количеством дней объявления, с данными по площади кухни и жилой площади. Количество этажей и название населенного пункта мы не можем сами придумать и таких пропусков минимальное значение 

Таким образом мы убрали из датафрейма самые большие пропуски

In [ ]:
#Вывод датафрейма для установления столбцов, где необходтио поменять тип данных
data.info()

**Изменение типов данных**

Тип столбца 'first_day_exposition' object, необходимо заменить на datetime, т.к. в этом столбце указан день публикации объявления. Также можно заменить на int64 значение в столбцах в которых по определению не может быть вещественного числа, например 'balcony' и 'floors_total'            

In [ ]:
#Замена типа данных в столбце 'first_day_exposition'
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format = '%Y-%m-%dT%H:%M:%S')

#Замена типов данных в столбцах 'balcony' и 'floors_total' 

data['balcony'] = data['balcony'].astype('int')

data['floors_total'] = data['floors_total'].astype('int')

In [ ]:
#Вывод датафрейма после замены типа данных
data.info()

**Устранение неявных дублей в названиях населенных пунктов**

In [ ]:
#Вывод уникальных значений в столбце 'locality_name'
data['locality_name'].unique()

**Вывод**

В столбце имеются неявные дубликаты. В названиях есть отличия "е" и "ё". Также есть разные обозначения одного и погоже типа населенного пункта, например "посёлок городского типа" и "городской посёлок".

In [ ]:
#Замена буквы "ё" на "е"

data['locality_name'] = data['locality_name'].str.replace('ё', 'е')

#Приведения к единоиу варианту названия типа населенного пункта "городской поселок" на "поселок городского типа"

data['locality_name'] = data['locality_name'].str.replace('городской поселок','поселок городского типа')


In [ ]:
data['locality_name'].unique()

**Вывод**

У части населенных пунктов отсутствуют наименования типа, но без дополнительных данных уточнить это невозможно

**Поиск аномальных значений во всех столбцах**

In [ ]:
data.describe()

В столбце 'last_price' слишком большие значения, можно попробовать посмотреть значения и определить на какое число можно сократить значение в столбцах


In [ ]:
#Вывод уникальных значений столбцв 'last_price'
data['last_price'].sort_values()

Можно поделить на 1000

In [ ]:
# Уменьшение значения на 1000
data['last_price']=data['last_price']/1000

In [ ]:
display(data['last_price'].describe())

In [ ]:
#Вывод уникальных значений в столбце'balcony'
data['balcony'].unique()

В столбце 'balcony' имеются явные аномалии, количество балконов редко превышает значение более двух. 2 балкона уже большая редкость. Убираем строки со значением более 3 балкона

In [ ]:
#Удаление аномалий и редких значений в столбце 'balcony'
data = data.query('balcony <=3')

In [ ]:
#вывод числового описания данных
display(data['rooms'].describe())

В столбце 'rooms' минимальное значение 0, такое невозможно, либо это может быть студия. Максимальное значение 19, это маловероятный случай. Возможно это какие-то нежилые помещения.

In [ ]:
#Построим диаграмму размаха
data.boxplot(column='rooms')

По гистограмме выведенной ранее видно, что значений более 5 комнат значительно меньше по сравнению с остальными. По диаграмме размаха видно, что выбросы начинаются со значения 6 комнат. Таким образом можно убрать редкие и выбивающиеся значения больше 7

In [ ]:
#Удаление аномалий и редких значений в столбце 'rooms'
data = data.query('rooms <=7')

In [ ]:
#вывод числового описания данных
display(data['ceiling_height'].describe())

В столбце 'ceiling_height' максимальное значения высоты потолка 32, а минимальное 1. Потолки не могут быть меньше 2.5 метров по жилищным нормам, а потолки больше 5 метров редко встречаются. Значения потолков 25 метров и выше, возможно объяснить ошибкой и это скорей всего вещественного число 2.5

In [ ]:
#Заменяем значение потолков 25 метров и выше 
data.loc[data['ceiling_height']>=25,'ceiling_height'] = data['ceiling_height']/10

#Удаление аномалий и редких значений в столбце 'ceiling_height'
data = data.loc[(data['ceiling_height']<=5) & (data['ceiling_height']>=2.5)| (data['ceiling_height'].isna())]

In [ ]:
display(data['rooms'].describe())

Рассмотрим столбец 'last_price'. Ранее мы его поделили на 1000

In [ ]:
#вывод числового описания данных столбца 'last_price'
display(data['last_price'].describe())

Большая часть значений лежит в диапазоне до 7.1 млн. рублей. Минимальная цена всего 12 тыс.рублей, а максимальная 330 млн. рублей. Среднее значение выше медианного почти на 2 млн. рублей, т.е. в имеются большие значения, которые влияют на среднее, но не влияют на медиану

In [ ]:
#Построим диаграмму размаха для столбца 'last_price'
plt.ylim(1000, 50000)
data.boxplot(column='last_price')
#Строим гистограмму для выяления количества больших значений
plt.show()
data['last_price'].hist(range=(1000, 60000))
#Строим гистограмму для выяления количества меньших значений
plt.show()
data['last_price'].hist(bins=100, range=(1, 10000))

Цена свыше 30 млн. рублей уже редко встречается, цена после 40 млн. рублей встречается ещё реже, а цена более 50 млн. рублей сливается с нулем, скорей всего далее уже единичные случаи. Для работы можно взять цену меньше 48 млн. рублей. Цена менее 1 млн. рублей также практически сливается с нулем, их тоже можно убрать

In [ ]:
#Удаление аномалий и редких значений в столбце 'last_price'
data = data.query('last_price <=48000 and last_price >= 1000')

In [ ]:
plt.ylim(1000, 25000)
data.boxplot(column='last_price')

In [ ]:
#вывод числового описания данных столбца 'total_area'
display(data['total_area'].describe())

Среднее значение выше медианного, значит, в показателях есть большие выбросы, которые сильно влияют на среднее, но не на медиану 

In [ ]:
#Построим диаграмму размаха для столбца 'total_area'
plt.ylim(1, 330)
data.boxplot(column='total_area')
#Строим гистограмму для выяления выдиляющихся значений
plt.show()
data['total_area'].hist(bins = 30, range=(1, 350))

По диаграмме размаха видно, что в столбце 'total_area' имеются выбросы превышающие значения после 100 кв.м., редкие выбросы свыше 200 кв. м. По гистограмме видно, что значения после 180 кв.м очень редки, после 260 практически равны 0. Также имеются редкие значения с площадью до 15 кв.м.  

In [ ]:
#Удаление аномалий и редких значений в столбце 'total_area'
data = data.query('total_area <=260 and total_area >=15')

In [ ]:
#вывод числового описания данных столбца 'living_area'
display(data['living_area'].describe())

In [ ]:
#Построим диаграмму размаха для столбцв 'living_area'
plt.ylim(1, 150)
data.boxplot(column='living_area')
#Строим гистограмму для выяления выдиляющихся значений
plt.show()
data['living_area'].hist(bins = 20, range=(1, 150))


По диаграмме размаха видно, что в столбце 'living_area' имеются выбросы превышающие значения после 80 кв.м. По гистограмме видно, что значения после 80 кв.м уменьшаются, а после 150 кв.м, практически равны 0. Также есть редкие значения с жилой площадью близкой к 0.

In [ ]:
#Удаление аномалий и редких значений в столбце 'living_area'
data = data.loc[(data['living_area']<=130) & (data['living_area']>=5)| (data['living_area'].isna())]

In [ ]:
display(data['kitchen_area'].describe())

In [ ]:
#Построим диаграмму размаха для столбцв 'living_area'
plt.ylim(1, 100)
data.boxplot(column='kitchen_area')
#Строим гистограмму для выяления выдиляющихся значений
plt.show()
data['kitchen_area'].hist(bins = 30, range=(1, 60))

По диаграмме размаха видно, что в столбце 'kitchen_area' имеются выбросы превышающие значения после 20 кв.м., есть отдельный выброс 60 кв. м. По гистограмме видно, что значения после 30 кв.м уменьшаются, а после 45 кв.м, практически равны 0. Также имеются редкие значения с площадью до 2 кв.м.

In [ ]:
#Удаление аномалий и редких значений в столбце 'kitchen_area'         
data = data.loc[(data['kitchen_area']<=45) & (data['kitchen_area']>=2)|(data['kitchen_area'].isna())]

In [ ]:
#вывод числового описания данных столбца 'floors_total'
display(data['floors_total'].describe())

Среднее значение выше медианного, значит в показателях есть большие выбросы, которые сильно влияют на среднее, но не на медиану 

In [ ]:
#Построим диаграмму размаха для столбцв 'floors_total'
plt.ylim(1, 100)
data.boxplot(column='floors_total')
#Строим гистограмму для выяления выдиляющихся значений
plt.show()
data['floors_total'].hist(bins = 30, range=(1, 70))

Значения после 30 практически равны 0, но есть редкие выбросы после 30 этажа

In [ ]:
#Удаление аномалий и редких значений в столбце 'floors_total'
data = data.query('floors_total <=35')

In [ ]:
#вывод числового описания данных столбца 'floor'
display(data['floor'].describe())

Среднее значение выше медианного, значит, в показателях есть выбросы, которые сильно влияют на среднее, но не на медиану. Учитывая, что в столбце 'floors_total' мы оставили значение меньше 35 этажей, а максимальное значение в столбце 'floor' - 33 этаж, то можно оставить столбец без изменений

In [ ]:
#вывод числового описания данных столбца 'days_exposition'
display(data['days_exposition'].describe())

In [ ]:
#Построим диаграмму размаха для столбцв 'days_exposition'
plt.ylim(10, 2000)
data.boxplot(column='days_exposition')
#Строим гистограмму для выяления выдиляющихся значений
plt.show()
data['days_exposition'].hist(bins = 30, range=(1, 70))


В диаграмме размаха выбросы в столбце 'days_exposition' начинаются после 500 дней. Среднее значение почти в 2 раза больше медианного, т.е. есть очень большие выбросы. В построенной ранее гистограмме заметно, что после 1200 дней, количество значений уменьшается и стремиться к нулю. Убираем значения больше 1200 дней

In [ ]:
#Удаление аномалий и редких значений в столбце 'days_exposition'
data = data.loc[(data['days_exposition']<=1200) | (data['days_exposition'].isna())]

По построенным ранее гистограммам видно, что в столбцах 'parks_around3000' и 'ponds_around3000' имеется всего по 4 значения. Расстояние в столбце 'parks_nearest' стремиться к 0 после значения 2500 м. 

In [ ]:
#Удаление аномалий в столбце 'parks_nearest'
data = data.loc[(data['parks_nearest']<=2500) | (data['parks_nearest'].isna())]

In [ ]:
#Построим диаграмму размаха для столбца 'total_images'
plt.ylim(1, 60)
data.boxplot(column='total_images')

Выбросы начинаются после значения в 25 фотографий.

In [ ]:
#Удаление аномалий и редких значений в столбце 'total_images'
data = data.query('total_images <=30')

In [ ]:
#вывод числового описания данных столбца 'airports_nearest'
display(data['airports_nearest'].describe())

В столбце 'airports_nearest' минимальное значение равно 0, этого не может быть, т.к. жилая недвижимость не может находиться на территории аэропорта

In [ ]:
#Построим диаграмму размаха для столбца 'airports_nearest'
plt.ylim(0, 85000)
data.boxplot(column='airports_nearest')

#Строим гистограмму для выяления количества меньших значений
plt.show()
data['airports_nearest'].hist(range=(0, 10000))

В диаграмме размаха по столбцу 'airports_nearest' выбросы начинаются с после расстояния приблизительно 60000 м. В построенной ранее гистограмме также видно, что после 75000 м значения стремиться  к 0. В гистограмме для выявления меньших значений видны редкие аномалии в районе 0 значения. Можно убрать всё, что ниже 6000 м.

In [ ]:
#Удаление аномалий и редких значений в столбце 'airports_nearest'
data = data.loc[(data['airports_nearest']<=75000) & (data['airports_nearest']>=6000)|(data['airports_nearest'].isna())]

In [ ]:
#вывод числового описания данных столбца 'city_centers_nearest'
display(data['city_centers_nearest'].describe())

In [ ]:
#Построим диаграмму размаха для столбца 'city_centers_nearest'
plt.ylim(0, 65000)
data.boxplot(column='city_centers_nearest')

По диаграмме размаха по столбцу 'city_centers_nearest' видно, что выбросы начинаются с расстояния в 25000 м. Также есть выбросы в районе 0, но их можно объяснить тем, что недвижимость находится непосредственно в центре города. По построенной ранее диаграмме видно, что после расстояния в 40000 м, значения становятся редкими, но есть небольшой всплеск на расстоянии 50000 м. После 60000 м. значения сравниваются с 0

In [ ]:
#Удаление аномалий и редких значений в столбце 'city_centers_nearest'
data = data.loc[(data['city_centers_nearest']<=58000)|(data['city_centers_nearest'].isna())]

In [ ]:
#Вывод дата фрейма после устранения редких и выбивающиеся значения во всех столбцах
data.describe()

In [ ]:
#Сброс индекса после удаления редких и выбивающиеся значения во всех столбцах
data = data.reset_index (drop=True)

In [ ]:
#Вывод общей информации о датафрейме
data.info()

In [ ]:
# Расчитываем долю удаленных значений
deleted_values = (23699-22333)/23699 * 100
deleted_values

Доля удаленных сообщений составила 5,76%

### Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
#Расчет и добавление столбца с ценой одного квадратного метра и перевод его в числовой тип.
#Умножаем на 1000, т.к. ранее цену делили на 1000
data['price_per_square_meter'] = data['last_price']/data['total_area']
data['price_per_square_meter'] = (data['price_per_square_meter']*1000).astype('int')

In [ ]:
#Расчет и добавление столбца с днем публикации объявления
data['week_day'] = data['first_day_exposition'].dt.weekday

In [ ]:
#Расчет и добавление столбца с месяцем публикации объявления
data['month'] = data['first_day_exposition'].dt.month

In [ ]:
#Расчет и добавление столбца с годом публикации объявления
data['year'] = data['first_day_exposition'].dt.year

In [ ]:
#Создаем функцию одной строки для определения типа этажа квартиры
def apartment_floor_type(row):# создайте функцию categorize_purpose()purpose
       
        if row['floor'] == 1:
            return 'Первый'
        elif row['floor'] == row['floors_total']:
            return 'Последний'        
    
        return 'Другой'

#Создание столбца с типом этажа квартиры
data['apartment_floor_type'] = data.apply(apartment_floor_type, axis=1)    

In [ ]:
#Расчет и добавление столбца с расстоянием до центра города в км.
data['distance_km_to_city_center'] = data['city_centers_nearest']/1000

In [ ]:
# Вывод датафрейма после добавления новых столбцов
data.head(20)

**Вывод по предобработке** После заполнения пропусков, удаления неявных дубликатов и выбивающиеся значений изначальная таблица сократилась на 1366 строк, что составляет 5,76%. Также рассчитали и добавили в таблицу шесть столбцов с новыми параметрами, необходимыми для дальнейшего анализа  

### Проведите исследовательский анализ данных

**Общая площадь**

In [ ]:
#Постройка гистограмм
data['total_area'].hist(bins=70)
plt.show()
data['total_area'].hist(bins=170, range=(0,50))

 **Вывод**

Большую часть объявлений (более 1000) занимают объявления с площадью менее от 30 кв.м до 50 кв.м., максимальное значение 45 кв.м. Количество объявлений с площадью более 75 кв.м постепенно снижается. После площади в 100 кв.м количество объявлений заметно падает, а после 150 кв.м стремится к 0   

**Жилая площадь**

In [ ]:
data['living_area'].hist(bins=70)
plt.show()
data['living_area'].hist(bins=170, range=(0,20))
plt.show()
data['living_area'].hist(bins=170, range=(20,50))

**Вывод**

Самое большое количество объявлений приходится на объявления, где жилая площадь составляет 18 кв.м. Также наблюдается увлечение объявлений с жилой площадью от 25 кв.м до 35 кв.м. Затем количество объявлений постепенно уменьшается, после 80 кв.м количество объявлений стремится к нулю

**Площадь кухни**

In [ ]:
#Постройка гистограмм
data['kitchen_area'].hist(bins=50)
plt.show()
data['kitchen_area'].hist(bins=150, range=(0,20))

**Вывод**

Самое большое количество объявлений с площадью кухни 6 и 10 кв.м. После площади кухни в 12 кв.м количество объявлений постепенно снижается. После 20 кв.м количество объявлений стремится к 0

**Цена обекта**

In [ ]:
#Постройка гистограмм
data['last_price'].hist(bins=120)
plt.show()
data['last_price'].hist(bins=120, range=(0,10000))

**Вывод**

Самое большое количество объявлений в диапазоне от 3 млн.руб до 5 млн. руб. После 10 млн. руб количество объявлений снижается, после 20 млн. количество объявлений стремится к 0

**Количество комнат**

In [ ]:
#Постройка гистограмм
data['rooms'].hist(bins=20)


**Вывод**

Самое большое количество объявлений приходится на однокомнатные квартиры, чуть меньше на двухкомнатные. Объявлений о трехкомнатных квартирах примерно на 2000 шт. меньше чем однокомнатных и двухкомнатных. Начиная с четырехкомнатных и далее,  количество объявлений постепенно снижается. Также есть объявления с 0 в графе количество комнат. Возможно это квартиры-студии


**Высота потолков**

In [ ]:
#Постройка гистограмм
data['ceiling_height'].hist(bins=30)

**Вывод**

Самое большое количество объявлений с высотой потолков 2.5 метра. Дальше заметно резкое снижение, до 2.75 метров, затем ещё одно снижение почти до 0 с резким всплеском на значении 3 метра

**Этаж квартиры**

In [ ]:
#Постройка гистограмм
data['floor'].hist(bins=35)
plt.show()
data['floor'].hist(bins=40, range=(0,15))


**Вывод**

Самое большое количество объявлений с 1 по 5 этаж (самое большое 2 этаж). Затем наблюдается постепенное снижение объявлений. После 25 этажа количество объявлений стремится к 0


**Тип этажа квартиры**

In [ ]:
#Постройка гистограмм
data['apartment_floor_type'].hist()


**Вывод**

Самое большое количество объявлений в категории 'Другие'. Количество в категориях 'Первый' и 'Последний' примерно одинаковое. 'Последних' чуть больше

**Общее количество этажей в доме**

In [ ]:
#Постройка гистограмм
data['floors_total'].hist(bins=20)
plt.show()
data['floors_total'].hist(bins=20, range = (0,10))

**Вывод**

Самое большое количество этажей в доме равно 5, затем идут дома в 9 этажей (самые распространенные по этажности типы домов в стране). Затем идет постепенное снижение с небольшим всплеском на 15 этажах. После 25 этажей количество объявлений минимально

**Расстояние до центра города**

In [ ]:
#Постройка гистограмм
data['city_centers_nearest'].hist(bins=50)
plt.show()
data['city_centers_nearest'].hist(bins=20, range = (0,20000))

**Вывод**

Самое большое количество объявлений о недвижимости находящейся на расстоянии 12 и 15 км. от центра Санкт-Петербурга. В целом основная масса объявлений о недвижимости находится в промежутке от 10 до 17 км. Это можно объяснить тем, что новые крупные микрорайоны как правило, строят на окраине городов, а также активно застраивается пригород. Также можно отметить большое количество в 5 км. от центра. После 17 км. отмечается снижение числа объявлений с небольшим всплеском в районе 30 км. Также можно отметить небольшое количество объявлений непосредственно в центре Санкт-Петербурга

**Расстояние до ближайшего аэропорт**

In [ ]:
#Постройка гистограмм
data['airports_nearest'].hist(bins=50)
plt.show()
data['airports_nearest'].hist(bins=20, range = (0,40000))

**Вывод**

Основная масса объявлений о недвижимости находящейся на расстоянии 15, 20 и 25 км от ближайшего аэропорта. В целом основная масса объявлений о недвижимости находится в промежутке от 10 до 25 км. После 25 км. отмечается снижение числа объявлений с достаточно большим всплеском в районе 33-37 км. Это соотносится со всплеском объявлений в столбце расстояние до 'центра города'. Можно предположить, что это крупные населенный пункт, находящийся на расстоянии в приблизительно 30 км. от Санкт-Петербурга.

**Расстояние до ближайшего парка**

In [ ]:
#Постройка гистограмм
data['parks_nearest'].hist(bins = 10)

**Вывод**

В большинстве объявлений указано отсутствие парков в близи объектов недвижимости. Из оставшихся объявлений чаще всего указано расстояние до ближайшего парка - до 500 м, после показателя в 750 м, заметно снижение количества объявлений с увеличением расстояния до ближайшего парка

**День публикации**


In [ ]:
#Постройка гистограмм
data['week_day'].hist(bins = 20)

**Вывод**

Большинство объявлений публикуются в будни (чаще всего во вторник и четверг). В выходные объявления публикуются почти в 2 раза реже чем в будни

**Месяц публикации**

In [ ]:
#Постройка гистограмм
data['month'].hist(bins = 30)

**Вывод**

Чаще всего объявления публикуют в феврале, затем в марте и апреле небольшой спад и резкий спад в мае, и затем небольшой рост летом. В летние месяцы количество объявлений практически не меняется. С начала осени от месяца к месяцу заметен рост числа объявлений, после чего резкий спад в декабре и январе. Спад объявлений в декабре и январе можно объяснить, новым годом. Перед новым годом как правила все стремиться закрыть сделки. Спад в мае можно объяснить майскими выходными, когда практически две недели никто не работает. Спад летом - период отпусков.   

**Как быстро продаются квартиры**

In [ ]:
#Постройка гистограмм
data['days_exposition'].hist(bins=100, range=(0, 1500))
plt.show()
data['days_exposition'].hist(bins=100, range = (1,100))
plt.show()
data['days_exposition'].hist(range = (50,500))

In [ ]:
plt.ylim(10, 1500)
data.boxplot(column='days_exposition')

In [ ]:
#Вывод числовых значений
display(data['days_exposition'].describe())

**Вывод**

Медианное значение равно 92, т.е. половина всех объявлений была размещена не более 94 дней. Среднее значение равно 174 дням. Это говорит о том, что имеются объявления, период размещения которых был очень большим, максимальное значение 1200 дней, т.е. 3 года. Минимальное значение 1 день, возможно объявления близкие к 1 были сняты в связи с какой-либо ошибкой и это не обязательно связанно с продажей. Заметны всплески 30, 45, 60, 90 и кажется 7 дней - всплески через ранее промежутки времени. Возможно, это какой-то технический сбой или правило. Учитывая это, при определении "быстрых продаж", я бы предложил ориентироваться на нижний квартиль(Q1), т.е первые 25% всех объявлений и считать такими продажами количество дней, которых меньше или равно 45. По гистограмме видно, что после всплеска на 90 днях идет резкое снижение до 250 дней, с последующим медленным затуханием. По диаграмме размаха выбросы начинаются после 500 дней. Учитывая это, при определении "необычно долгих продаж", я бы предложил ориентироваться на верхний квартиль(Q3) и считать такими продажами количество дней, которых больше или равно 230, т.е. последние 25% всех объявлений

**Выявление факторов больше всего влияющих на общую (полную) стоимость объекта**

**Площадь обекта**

In [ ]:
#Строим сводную таблицу для значений площади объекта и его цены
pivot_area_price = data.pivot_table(index = 'total_area', values = 'last_price')

#Строим диаграмму рассеяния 
pivot_area_price.plot(y = 'last_price', style = 'o')

#Считаем коэффициент корреляции Пирсона
data['total_area'].corr(data['last_price'])



**Вывод**

Коэффициент Пирсона для цены объекта и его площади составляет 0.768, т.е. значение близко к 1, значит, площадь объекта достаточно сильно влияет на его стоимость. Это же подтверждается диаграммой рассеяния. По диаграмме видно, что основные выбросы появляются с увеличением значений. Есть отдельные выбросы на небольших значениях цены и площади, это возможно объяснить, например географической близостью объекта к центру города, когда например небольшой объект площадью 40 кв., может продаваться за 25 млн. руб. и наоборот большие по площади объекты могут стоить до 10 млн. рублей, но находиться очень далеко от города

**Жилая площадь**

In [ ]:
#Строим сводную таблицу для значений жилой площади объекта и его цены
pivot_living_price = data.pivot_table(index = 'living_area', values = 'last_price')

#Строим диаграмму рассеяния 
pivot_living_price.plot(y = 'last_price', style = 'o')

#Считаем коэффициент корреляции Пирсона
data['living_area'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для цены объекта и жилой площади объекта составляет 0.661, значение чуть меньше, чем у общей площади, соответственно жилая площадь объекта влияет на его стоимость чуть меньше, чем общая. Диаграмма рассеяния для жилой площади в целом похожа на диаграмму рассеяния для общей площади, на ней даже видны тоже самые выбросы. Также на коэффициент могли повлиять пропуски в столбце 'living_area'

**Площадь кухни**

In [ ]:
#Строим сводную таблицу для значений площади кухни объекта и его цены
pivot_kitchen_price = data.pivot_table(index = 'kitchen_area', values = 'last_price')

#Строим диаграмму рассеяния 
pivot_kitchen_price.plot(y = 'last_price', style = 'o')

#Считаем коэффициент корреляции Пирсона
data['kitchen_area'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для цены объекта и площади кухни объекта составляет 0.578, значение еще меньше, чем у жилой площади, по сути, получаем среднею зависимость цены от площади кухни. По диаграмме рассеяния сильно корригируются значения при небольшой площади кухни, после значения в 20 кв.м зависимость ослабевает

**Количество комнат**

In [ ]:
#Строим сводную таблицу для значений количество комнат в объекте и его цены
pivot_rooms_price = data.pivot_table(index = 'rooms', values = 'last_price')

#Строим диаграмму
pivot_rooms_price.plot()

#Считаем коэффициент корреляции Пирсона
data['rooms'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для количества комнат в объекте и его цены составляет 0.474, что говорит о слабой зависимости цены объекта от количества комнат в нем. По графику видно, что цена в зависимости от количества комнат в целом возрастает, это можно объяснить тем, что чем больше комнат, тем соответственно больше общая площадь.


**Тип этажа**

In [ ]:
#Строим сводную таблицу для значений типа этажа объекта и его цены и рассчитываем средную цену для каждого типа этажа
pivot_floor_type_price = data.pivot_table(index = 'apartment_floor_type', values = 'last_price', aggfunc = ['mean'])

#Дабавляем столбец со средним значением и выводим результат
pivot_floor_type_price.columns = ['mean']
pivot_floor_type_price.plot()
pivot_floor_type_price


**Вывод**

По диаграмме видно, что самые дешевые объекты находятся на Первом этаже. Это же подтверждается значением цены объекта для данного типа. Средняя цена объекта расположенного на Последнем этаже почти на 20% выше средней цены объекта расположенного на Первом этаже. Стоимость объектов имеющих тип этажа Другие почти на 30% выше стоимости объектов на первом этаже. Таким образом Самые дешевые объекты расположены на Первом этаже, саме дорогие имеют тип этажа Другие, т.е. тип этажа влияет на стоимость объекта  

**Дата размещения. День**

In [ ]:
#Строим сводную таблицу для значений день размещения объявления и цены объекта
pivot_day_price = data.pivot_table(index = 'week_day', values = 'last_price')

#Строим диаграмму 
pivot_day_price.plot()

#Считаем коэффициент корреляции Пирсона
data['week_day'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для цены объекта и дня размещения объявления составляет - 0.016, т.е. имеем очень слабую отрицательную корреляцию. Нельзя сделать вывод о какой-либо зависимости цены объекта от дня размещения объявления. Если сравнить с гистограммой по количеству объявлений в день недели, то можно заметить, что самая высокая цена выставляется в четверг, в день когда размещается больше всего объявлений, а на выходные приходится самая низкая цена, так же как и количество объявлений

**Дата размещения. Месяц**

In [ ]:
#Строим сводную таблицу для значений месяц размещения объявления и цены объекта
pivot_month_price = data.pivot_table(index = 'month', values = 'last_price')

#Строим диаграмму 
pivot_month_price.plot()

#Считаем коэффициент корреляции Пирсона
data['month'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для цены объекта и месяца размещения объявления составляет - 0.002, т.е. коэффициент практически равен 0. Цена объекта и месяц размещения никак не связанны. Если сравнить с гистограммой по количеству объявлений в месяц, то можно заметить, что в мае, когда меньше всего объявлений, практически самые высокие цены, а в октябре наоборот - большое количество объявлений и просто провал по ценам. Возможно, это связанно с большим количеством объявлений о продаже загородной недвижимости и окончанием летнего сезона.

**Дата размещения. Год**

In [ ]:
#Строим сводную таблицу для значений год размещения объявления и цены объекта
pivot_year_price = data.pivot_table(index = 'year', values = 'last_price')

#Строим диаграмму 
pivot_year_price.plot()

#Считаем коэффициент корреляции Пирсона
data['year'].corr(data['last_price'])

**Вывод**

Коэффициент Пирсона для цены объекта и года размещения объявления составляет - 0.031, т.е. имеем очень слабую отрицательную корреляцию. Нельзя сделать вывод о какой-либо зависимости цены объекта от года размещения объявления. По диаграмме можно заметить, что самые высокие цены были в 2014 году, потом последовал резкий спад с плавным снижением до 2018 года, после чего начался рост стоимости. Возможно в данных за 2014 год есть какие-то ошибки. либо в этот год был выставлен на продажу самый дорогой объект

**Средняя цена одного квадратного метра в 10 населённых пунктах**

In [ ]:
#Строим сводную таблицу для значений наименование населенного пункта и цены одного квадратного метра и считаем количество объявлений и средную цену
pivot_table_locality = data.pivot_table(index = 'locality_name', values = 'price_per_square_meter', aggfunc=['count', 'mean'])

#Добавляем в сводную таблицу столбцы 'count' и 'mean'
pivot_table_locality.columns = ['count', 'mean']

#Сортируем таблицу по убыванию количества объявлений в населенном пункте и сохраняем первые 10 значений
pivot_table_locality = pivot_table_locality.sort_values('count', ascending = False).head(10)
pivot_table_locality


In [ ]:
#Вывод самой высокой стоимости
pivot_table_locality[pivot_table_locality['mean']==pivot_table_locality['mean'].max()]


In [ ]:
#Вывод самой низкой стоимости
pivot_table_locality[pivot_table_locality['mean']==pivot_table_locality['mean'].min()]

**Вывод**
Подавляющее большинство объявлений о продаже недвижимости в Санкт-Петербурге (14881), объявлений больше почти в 30 раз по сравнению с поселком Мурино (537), находящимся на 2 месте по количеству объявлений. Меньше всего объявлений в Топ 10 по количеству объявлений у Выборга (230). Тоже самое со стоимостью одного квадратного метра. В Санкт-Петербурге самая высокая средняя цена - 111618 руб, самая низкая в Топ 10 в Выборге - 58843 руб.   

**Средняя цена киломметра в Санкт-Петербурге** 

In [ ]:
# заменяем пропуски в столбце 'distance_km_to_city_center', для этого заменяем пропуски на число которого не может быть в растоянии
data['distance_km_to_city_center'] = data['distance_km_to_city_center'].fillna(100000)

#Перевод значений в столбце 'distance_km_to_city_center' в тип 'int' для округления до целого числа значения км.
data['distance_km_to_city_center'] = data['distance_km_to_city_center'].astype('int')

#Создаем сводную таблицу по срезу значений Санкт-Петербург, за исключением значений добавленных для заполнения пропуска
pivot_table_km_to_city = data.query('locality_name == "Санкт-Петербург" and distance_km_to_city_center !=100000').pivot_table(index = 'distance_km_to_city_center', values = 'price_per_square_meter', aggfunc = 'mean')

#Постройка диаграммы
pivot_table_km_to_city.plot()

#Вывод сводной таблицы
pivot_table_km_to_city

**Вывод**

По диаграмме видно, что чем ближе расположены объекты к центру города, там выше цена кв. м. На расстоянии 2 и 3 км от центра города цена кв.м ниже чем на расстоянии от 4 до 7 км. Возможно, это как-то связанно с географическими особенностями. После 7 км цена за кв.м постепенно снижается по мере удаления, но есть несколько всплесков на 20,23 и 27 км. Цена недвижимости на 27 км сопоставима со стоимость кв.м. в центре Санкт-Петербурга. Возможно, предположить, что на этих расстояниях находится элитные населенные пункты. Можно предположить, что есть зависимость стоимость кв.м. от расстояния до центра города, но встречаются серьезные аномальные всплески, которые выбиваются из общей динамики

### Общий вывод

Нами проведен исследовательский анализ архива объявлений с сервиса Яндекс Недвижимость о продаже квартир в городе Санкт-Петербурге и прилагающих районах. На этапе предобработки данных было проанализировано содержание предоставленной таблицы, выявлены пропуски в столбцах и заменены, там, где это было возможным, на значения, минимально влияющие на последующие расчеты. Проанализированы типы данных и произведена замена типа данных на нужный тип в тех категориях, которые необходимы были для последующего анализа. Выявлены неявные дубликаты в названиях населенных пунктов - буква "е" и "ё" в названиях, а так же похожие типы названий "городской поселок" и "поселок городского типа". Эти дубли были приведены к единой форме. С помощью метода describe(), гистограмм и диаграмм размаха проанализированы данные по всем столбцам и устранены редкие и аномальные значения. В таблицу были добавлены столбцы с ценой одного квадратного метра, с денем, месяцем и годом публикации, определен тип этажа квартиры и расстояние до центра города.

Изучены следующие параметры:
**Общая площадь** - Чаще всего на продажу выставляют объекты с общей площадью от 30 кв.м до 50 кв.м., максимальное значение 45. кв.м. Затем общая площадь продаваемых объектом постепенно снижается. Рассчитав Коэффициент Пирсона для цены объекта и его площади (0.768), **установлена сильная зависимость** цены квартиры от её площади. На диаграмме рассеяния видны отдельные выбросы, но в целом видно, что цена увеличивается по мере увеличения общей площади квартиры;

**Жилая площадь** - Самые частые объявления о продаже квартир с жилой площадью в 18 кв.м. - скорей всего это однокомнатные квартиры. Также много объявлений с жилой площадью от 25 кв.м до 35 кв.м. Затем количество объявлений постепенно снижается. В целом можно наблюдать похожую динамику с общей площадью, что подтверждается коэффициентом Пирсона (0.661), но цена квартиры **чуть менее зависима** от размера жилой площади;

**Площади кухни** - Самые частые объявлений с площадью кухни 6 и 10 кв.м. После площади кухни в 12 кв.м количество объявлений постепенно снижается. Так же можно предположить, что это однокомнатные квартиры. В целом динамика также похожа на значения с общей и жилой площадью, но **показывает среднее влияние на цену**, чуть слабее, чем у предыдущих показателей. Коэффициент  Пирсона - 0.578;

**Количество комнат** - Как и предполагалось ранее самое большое количество объявлений это однокомнатные квартиры, чуть меньше объявлений на двухкомнатные. Затем с увеличением количества комнат, снижается количество объявлений. Коэффицент Пирсона (0.474) говорит о **слабой зависимости** цены объекта от количества комнат, но она есть, это можно объяснить тем что чем больше комнат, тем соответственно больше площадь;

**Типа этажа** - Самое большое количество объявлений в категории 'Другие' этажи, т.е. не первый и не последний этаж. Количество объявлений в категориях 'Первый' и 'Последний' примерно одинаковое. Самые дешевые объекты находятся на Первом этаже. Средняя цена объекта расположенного  на Последнем этаже почти на 20% выше средней цены объекта расположенного на Первом этаже. Стоимость объектов имеющих тип этажа Другие почти на 30% выше стоимости объектов на первом этаже. Таким образом, **тип этажа влияет на стоимость объекта**; 

**Даты размещения** - Коэффициент  Пирсона для дня, месяца и года составляет -0.016, -0.002 и -0.031, т.е. имеем очень слабую отрицательную корреляцию, значит, **дата публикации объявления не влияет на цену объекта**. Большинство объявлений публикуются в будни (чаще всего во вторник и четверг). В выходные объявления публикуются почти в 2 раза реже, чем в будни. Можно отметить, в мае, когда меньше всего объявлений, практически самые высокие цены, а в октябре наоборот - большое количество объявлений и просто провал по ценам. Возможно, это связанно с большим количеством объявлений о продаже загородной недвижимости и окончанием летнего сезона. Чаще всего объявления публикуют в феврале, затем в марте и апреле. Летом количество объявлений снижается, вероятно, в связи с периодом отпусков. Самые высокие цены были в 2014 году, потом последовал резкий спад с плавным снижением до 2018 года, после чего начался рост стоимости. Возможно в данных за 2014 год есть какие-то ошибки. либо в этот год был выставлен на продажу самый дорогой объект или сразу несколько объектов;

**Время продажи квартир** - Половина всех объявлений была размещена на сайте не более 94 дней. В среднем объявление находилось на сайте 174 дня. Можно сделать вывод, что имеются объявления, период размещения которых был очень большим, максимальное значение 1200 дней. Имеются объявления, находившиеся на сайте 1 день и чуть более, возможно это связано с какой-либо ошибкой в размещении объявления. Заметны всплески 30, 45, 60, 90 и кажется 7 дней - всплески через равные промежутки времени. Возможно, это какой-то технический сбой или какое либо правило сайта. Быстрыми продажами являются продажи количество дней, которых меньше или равно 45, необычно долгими продажами более 230 дней;

**Цена одного квадратного метра** - В Санкт-Петербурге самая высокая средняя цена - 111618 руб, самая низкая в Топ 10 в Выборге - 58843 руб.;

**Средняя цена километра** - Чем ближе расположены объекты к центру города, там выше цена кв. м. Самые высокие цены находятся в радиусе 1 км. до центра. На расстоянии 2 и 3 км от центра города цена кв.м ниже чем на расстоянии от 4 до 7 км. Возможно, это как-то связанно с географическими особенностями. После 7 км цена за кв.м постепенно снижается по мере удаления, но есть несколько всплесков на 20,23 и 27 км. Цена недвижимости на 27 км сопоставима со стоимость кв.м. в центре Санкт-Петербурга. Возможно, что на этих расстояниях находится элитные населенные пункты. Можно предположить, что **есть зависимость стоимость кв.м. от расстояния до центра города**.

Таким образом, после проведения исследовательского анализа архива объявлений о продаже квартир в городе Санкт-Петербурге, выявлены зависимости стоимости объектов недвижимости от площади объекта, типа этажа на котором расположен объект и от расстояния до центра города Санкт-Петербург. Самые дорогие объявления у квартир находящихся в радиусе 1 километра от центра Санкт-Петербурга, расположенных не на первом и не на последнем этаже дома и с высокой общей площадью. Также можно отметить Высокие цены на квартиры в элитных поселках.



**Чек-лист готовности проекта**

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл
- [x]  файлы изучены (выведены первые строки, метод `info()`, гистограммы и т.д.)
- [x]  определены пропущенные значения
- [x]  заполнены пропущенные значения там, где это возможно
- [x]  есть пояснение, какие пропущенные значения обнаружены
- [x]  изменены типы данных
- [x]  есть пояснение, в каких столбцах изменены типы и почему
- [x]  устранены неявные дубликаты в названиях населённых пунктов
- [x]  устранены редкие и выбивающиеся значения (аномалии) во всех столбцах
- [x]  посчитано и добавлено в таблицу: цена одного квадратного метра
- [x]  посчитано и добавлено в таблицу: день публикации объявления (0 - понедельник, 1 - вторник и т.д.)
- [x]  посчитано и добавлено в таблицу: месяц публикации объявления
- [x]  посчитано и добавлено в таблицу: год публикации объявления
- [x]  посчитано и добавлено в таблицу: тип этажа квартиры (значения — «первый», «последний», «другой»)
- [x]  посчитано и добавлено в таблицу: расстояние в км до центра города
- [x]  изучены и описаны следующие параметры:
        - общая площадь;
        - жилая площадь;
        - площадь кухни;
        - цена объекта;
        - количество комнат;
        - высота потолков;
        - этаж квартиры;
        - тип этажа квартиры («первый», «последний», «другой»);
        - общее количество этажей в доме;
        - расстояние до центра города в метрах;
        - расстояние до ближайшего аэропорта;
        - расстояние до ближайшего парка;
        - день и месяц публикации объявления
- [x]  построены гистограммы для каждого параметра
- [x]  выполнено задание: "Изучите, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней «висело» каждое объявление.
    - Постройте гистограмму.
    - Посчитайте среднее и медиану.
    - В ячейке типа markdown опишите, сколько обычно занимает продажа. Какие продажи можно считать быстрыми, а какие — необычно долгими?"
- [x]  выполнено задание: "Какие факторы больше всего влияют на общую (полную) стоимость объекта? Постройте графики, которые покажут зависимость цены от указанных ниже параметров. Для подготовки данных перед визуализацией вы можете использовать сводные таблицы."
        - общей площади;
        - жилой площади;
        - площади кухни;
        - количество комнат;
        - типа этажа, на котором расположена квартира (первый, последний, другой);
        - даты размещения (день недели, месяц, год);
- [x]  выполнено задание: "Посчитайте среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделите населённые пункты с самой высокой и низкой стоимостью квадратного метра. Эти данные можно найти по имени в столбце `locality_name`."
- [x]  выполнено задание: "Ранее вы посчитали расстояние до центра в километрах. Теперь выделите квартиры в Санкт-Петербурге с помощью столбца `locality_name` и вычислите среднюю цену каждого километра. Опишите, как стоимость объектов зависит от расстояния до центра города."
- [x]  в каждом этапе есть промежуточные выводы
- [x]  есть общий вывод